In [60]:
# Install required packages if not already installed
!pip install --upgrade gspread oauth2client ipywidgets

# Import necessary libraries
import re
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import time
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.colab import userdata

from google.colab import output
output.enable_custom_widget_manager()

In [61]:
cfg=json.loads(userdata.get('gsheets_config_json'))
gsheets_uids={
    "screenshots":userdata.get("gsheets_screenshots_uid"),
    "annotations":userdata.get("gsheets_annotations_uid")
}
print(gsheets_uids["screenshots"])
cfg.keys()

16fzIloryNMivAgx6vaVkutakMKkvhjpEKMzR-UMXw4U


dict_keys(['type', 'project_id', 'private_key_id', 'private_key', 'client_email', 'client_id', 'auth_uri', 'token_uri', 'auth_provider_x509_cert_url', 'client_x509_cert_url', 'universe_domain'])

In [62]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = Credentials.from_service_account_info(cfg, scopes=SCOPES)
creds

gc = gspread.authorize(creds)
gsheet_screenshots=gc.open_by_key(gsheets_uids["screenshots"])
worksheet=gsheet_screenshots.get_worksheet(0)

screenshots_df = pd.DataFrame(worksheet.get_all_records())
screenshots_df

,image,sample_id,coordinates,REF,ALT,variant,path
0,SNV_3_300x_X_89189133,pair1,chrX:89189133,C,T,SNV,https://drive.google.com/uc?export=view&id=1WS...
1,SNV_3_300x_Y_59006690,pair1,chrY:59006690,C,A,SNV,https://drive.google.com/uc?export=view&id=1U5...
2,SNV_3_300x_Y_17215875,pair1,chrY:17215875,G,A,SNV,https://drive.google.com/uc?export=view&id=1Wk...
3,SNV_3_300x_X_141985081,pair1,chrX:141985081,G,T,SNV,https://drive.google.com/uc?export=view&id=1_T...
4,SNV_3_300x_X_92259539,pair1,chrX:92259539,G,T,SNV,https://drive.google.com/uc?export=view&id=1T-...
...,...,...,...,...,...,...,...
70,SNV_3_300x_1_38767400,pair1,chr1:38767400,T,C,SNV,https://drive.google.com/uc?export=view&id=1Nb...
71,SNV_3_300x_2_51725950,pair1,chr2:51725950,T,G,SNV,https://drive.google.com/uc?export=view&id=1uG...
72,SNV_3_300x_1_121472660,pair1,chr1:121472660,A,G,SNV,https://drive.google.com/uc?export=view&id=1-O...
73,SNV_3_300x_2_24686755,pair1,chr2:24686755,G,A,SNV,https://drive.google.com/uc?export=view&id=1le...


In [96]:
def get_not_yet_voted_candidates(
    training_questions,
    voting_institute,
    screenshots_df,
    n_sample=10):
    """
    Parameters:
      training_questions: int, number of training questions per centre to skip.
      voting_institute: string, the institute that is currently voting.
      screenshots_df: pandas DataFrame with columns: image, coordinates, path, REF, ALT, variant.
      n_sample: int, number of candidate rows to sample.

    Returns:
      A pandas DataFrame of candidate screenshots.
    """
    # Read annotations from Google Sheet
    sheets = gc.open_by_key(gsheets_uids["annotations"])
    worksheet = sheets.get_worksheet(0)
    annot_records = worksheet.get_all_records()
    annot = pd.DataFrame(annot_records)

    # Remove first `training_questions` rows per institute (simulate group_by(institute) and slice)
    if training_questions > 0:
        annot = annot.sort_values("timestamp")  # ensure a consistent order if needed
        annot = annot.groupby("institute", group_keys=False).apply(lambda df: df.iloc[training_questions:])

    # Create additional rows with image "-" and all agreement categories
    agreement_categories = ["yes", "no", "diff_var", "not_confident"]
    extra = pd.DataFrame({
        "image": "-",
        "already_voted": True,
        "agreement": agreement_categories
    })

    # Append extra rows to annotation dataframe
    annot = pd.concat([annot, extra], ignore_index=True)

    # For each annotation, mark whether the institute equals the voting institute
    annot["voted_by_institute"] = annot["institute"].apply(lambda x: x == voting_institute if pd.notnull(x) else False)

    # Group by image and agreement to count votes.
    # First, per image, determine if already voted by the voting_institute.
    def compute_already_voted(group):
        group = group.copy()
        group["already_voted"] = group["voted_by_institute"].any()
        return group

    annot = annot.groupby("image", group_keys=False).apply(compute_already_voted)

    # Count the votes per image, already_voted, and agreement.
    counts = annot.groupby(["image", "already_voted", "agreement"]).size().reset_index(name="n")

    # Pivot so each agreement type becomes a separate column.
    pivot = counts.pivot_table(index=["image", "already_voted"], columns="agreement", values="n", fill_value=0).reset_index()

    # Compute total_votes as the sum of all agreement counts.
    pivot["total_votes"] = pivot.get("yes", 0) + pivot.get("no", 0) + pivot.get("diff_var", 0) + pivot.get("not_confident", 0)

    # Merge candidate annotations with screenshots based on the "image" column.
    # Use a left join so that all screenshots are kept.
    candidates = screenshots_df.merge(pivot, on="image", how="left")

    # Replace NaN values: if an agreement column is missing, set it to 0.
    for col in ["yes", "no", "diff_var", "not_confident", "total_votes"]:
        if col in candidates.columns:
            candidates[col] = candidates[col].fillna(0)
        else:
            candidates[col] = 0

    # For the already_voted flag, fill missing with False.
    candidates["already_voted"] = candidates.get("already_voted", pd.Series([False]*len(candidates))).fillna(False)

    # Remove candidates where the institute already voted.
    candidates = candidates[candidates["already_voted"] == False]

    # Apply filtering rules: remove images with strong "yes" votes or strong "no" votes.
    # Note: To avoid division by zero, ensure total_votes is > 0.
    def filter_rule(row):
        if row["total_votes"] > 0:
            if (row["yes"] >= 3 and (row["yes"] / row["total_votes"]) > 0.7):
                return False
            if (row["no"] >= 3 and (row["no"] / row["total_votes"]) > 0.7):
                return False
        return True

    candidates = candidates[candidates.apply(filter_rule, axis=1)]

    # Subset a sample of screenshots
    if n_sample is not None:
        n_sample = min(len(candidates), n_sample)
        candidates = candidates.sample(n=n_sample) if n_sample > 0 else candidates

    # Replace the path to the image using regex
    # Replace "drive.google.com/uc?export=view&id=" with "lh3.googleusercontent.com/d/"
    candidates["path"] = candidates["path"].apply(lambda p: re.sub(r"drive\.google\.com/uc\?export=view&id=",
                                                                      "lh3.googleusercontent.com/d/", p)
                                                  if isinstance(p, str) else p)
    print("cand")
    return candidates

In [98]:
training_questions=10
voting_institute="Training (answers won't be saved)"
screenshots_df=screenshots_df
n_sample=10

get_not_yet_voted_candidates(
    training_questions,
    voting_institute,
    screenshots_df,
    n_sample
)

cand


<ipython-input-96-2ee4e4950923>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annot = annot.groupby("institute", group_keys=False).apply(lambda df: df.iloc[training_questions:])
<ipython-input-96-2ee4e4950923>:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annot = annot.groupby("image", group_keys=False).apply(compute_already_voted)


,image,sample_id,coordinates,REF,ALT,variant,path,already_voted,,diff_var,no,not_confident,yes,total_votes
54,SNV_3_300x_5_108367192,pair1,chr5:108367192,T,A,SNV,https://lh3.googleusercontent.com/d/1rDgt6LO-p...,False,0.0,0.0,0.0,0.0,1.0,1.0
73,SNV_3_300x_2_24686755,pair1,chr2:24686755,G,A,SNV,https://lh3.googleusercontent.com/d/1lek2yr-hM...,False,0.0,0.0,1.0,0.0,0.0,1.0
64,SNV_3_300x_4_20403924,pair1,chr4:20403924,G,T,SNV,https://lh3.googleusercontent.com/d/1ODn-rWAL2...,False,0.0,0.0,0.0,1.0,0.0,1.0
51,SNV_3_300x_4_181085112,pair1,chr4:181085112,T,C,SNV,https://lh3.googleusercontent.com/d/1h7Inf1O1a...,False,0.0,0.0,1.0,0.0,0.0,1.0
7,SNV_3_300x_21_11027202,pair1,chr21:11027202,A,G,SNV,https://lh3.googleusercontent.com/d/1voYU8hJQb...,False,0.0,0.0,0.0,0.0,1.0,1.0
60,SNV_3_300x_3_82117028,pair1,chr3:82117028,C,T,SNV,https://lh3.googleusercontent.com/d/1GFKebxO1n...,False,0.0,0.0,0.0,0.0,1.0,1.0
65,SNV_3_300x_3_70119872,pair1,chr3:70119872,C,G,SNV,https://lh3.googleusercontent.com/d/1gVqQseAf-...,False,0.0,0.0,0.0,0.0,1.0,1.0
43,SNV_3_300x_9_80150285,pair1,chr9:80150285,C,T,SNV,https://lh3.googleusercontent.com/d/1X9_B93MBW...,False,0.0,0.0,0.0,0.0,1.0,1.0
59,SNV_3_300x_4_140478484,pair1,chr4:140478484,A,G,SNV,https://lh3.googleusercontent.com/d/147CRnTeRp...,False,0.0,0.0,0.0,0.0,1.0,1.0
41,SNV_3_300x_8_27599063,pair1,chr8:27599063,C,T,SNV,https://lh3.googleusercontent.com/d/1BLMtxdZzK...,False,0.0,0.0,0.0,1.0,0.0,1.0


In [158]:
img_widget = widgets.Image(
    format='png',
    width=300,
    height=300
)

vote_options = {
    "Yes, it is.": "yes",
    "There is no variant.": "no",
    "There is a different variant.": "diff_var",
    "I'm not sure.": "not_confident"
}
vote_radio = widgets.RadioButtons(
    options=vote_options,
    description="Is variant correct?"
)

next_button = widgets.Button(
    description="Next",
    button_style='primary'
)
vote_out = widgets.Output()

voting_ui = widgets.VBox([
    img_widget,
    vote_radio,
    next_button,
    vote_out
])

voting_ui

In [161]:
# Global variable to store candidate pictures.
pic = pd.DataFrame()

def get_next_picture(
    training_questions,
    voting_institute,
    screenshots_df,
    n_sample=10
  ):
    print("called")
    global pic
    # If pic is empty, retrieve a new batch of pictures.
    if pic.empty:
        pic = get_not_yet_voted_candidates(
            training_questions,
            voting_institute,
            screenshots_df,
            n_sample=10
        )
        # If choose_picture returned an empty DataFrame, create a default row.
        if pic.empty:
            pic = pd.DataFrame({
                "image": ["done"],
                "REF": ["-"],
                "ALT": ["-"],
                "coordinates": ["There are no more variants to vote in this category!"],
                "path": ["https://imgpile.com/images/Ud9lAi.jpg"]
            })

    # Get the first row as a one-row DataFrame.
    first_pic = pic.iloc[0:1].copy()
    # Remove the first row from pic.
    pic = pic.iloc[1:].reset_index(drop=True)

    return first_pic

In [162]:
get_next_picture(training_questions, voting_institute, screenshots_df)

called
cand


<ipython-input-96-2ee4e4950923>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annot = annot.groupby("institute", group_keys=False).apply(lambda df: df.iloc[training_questions:])
<ipython-input-96-2ee4e4950923>:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annot = annot.groupby("image", group_keys=False).apply(compute_already_voted)


,image,sample_id,coordinates,REF,ALT,variant,path,already_voted,,diff_var,no,not_confident,yes,total_votes
51,SNV_3_300x_4_181085112,pair1,chr4:181085112,T,C,SNV,https://lh3.googleusercontent.com/d/1h7Inf1O1a...,False,0.0,0.0,1.0,0.0,0.0,1.0


In [166]:
import requests

img_widget = widgets.Image(
    format='png',
    width=1000,
    height=1000
)

vote_options = {
    "Yes, it is.": "yes",
    "There is no variant.": "no",
    "There is a different variant.": "diff_var",
    "I'm not sure.": "not_confident"
}
vote_radio = widgets.RadioButtons(
    options=vote_options,
    description="Is variant correct?"
)

next_button = widgets.Button(
    description="Next",
    button_style='primary'
)

voting_ui = widgets.VBox([
    img_widget,
    vote_radio,
    next_button,
])

display(voting_ui)
def update_vote_ui():
    """Update the voting UI with the next picture."""
    df_row = get_next_picture(
        training_questions,
        voting_institute,
        screenshots_df
    )
    url=df_row["path"][0]
    print(url)
    response = requests.get(url)
    img_widget.value=response.content

# Register the callback once (do not register it inside update_vote_ui)
def on_button_next(b):
    print("here")
    update_vote_ui()

next_button.on_click(on_button_next)

# Initially load the first image
update_vote_ui()

called
https://lh3.googleusercontent.com/d/18SbcmzWrW8WOKydMRlTgKAIMczf5Gr1-
here
called
https://lh3.googleusercontent.com/d/1k_5-nYSts3vAEBIRT379k6lDNxO_JAFB
here
called
https://lh3.googleusercontent.com/d/1TYUhzhA7NUQFQTP_FAR9igQW97m9ArXM
here
called
https://lh3.googleusercontent.com/d/147CRnTeRpnx5JIdZ7Jc7hIVcEa3sM6KB
here
called
https://lh3.googleusercontent.com/d/1voYU8hJQbir38-NteCP6cxBjHhTtrUcn
here
called
https://lh3.googleusercontent.com/d/11vZDswo85IBNnKwMYKIoCEuUQPIfS5fr


In [70]:
# Prompt the user for their identity (username)
user = input("Enter your username: ")
institute = input ("Enter your institute ")

Enter your username: test
Enter your institute Training
